# <center><font> Tutorial #2 Computer- and robot-assisted surgery</font></center>
## <center><font> Computer Vision Basics I</font></center>
<center>&copy; Sebastian Bodenstedt, National Center for Tumor Diseases (NCT) Dresden<br>
    <a href="https://www.nct-dresden.de/"><img src="https://www.nct-dresden.de/++theme++nct/images/logo-nct-en.svg"></a> </center>

## <center><font>Preperation</font></center>

For this tutorial, we will utilize the OpenCV, Matplotlib and NumPy:

In [ ]:
# Install numpy, opencv and matplot-lib pip packages into the current Jupyter kernel
import sys

In [ ]:
import cv2
import numpy as np
# Force Matplotlib to display data directly in Jupyter
%matplotlib inline 
from matplotlib import pyplot as plt

We will also download and extract a few images:

In [ ]:
import urllib.request
from os.path import basename, exists
import zipfile

def download_and_extract(url): #download and extract Zip archive
    file_path = basename(url)
    if not exists(file_path): # does zip file already exist?
        urllib.request.urlretrieve(url, file_path) # if not, download it
        with zipfile.ZipFile(file_path, 'r') as zip_ref: # and unzip it
            zip_ref.extractall(".")

In [ ]:
download_and_extract("http://tso.ukdd.de/crs/Exercise1.zip") # In case you didn't download the data last week

We now list the extracted files:

In [ ]:
!dir *

## <center><font color=navy>Review</font></center>
We can utilize OpenCV to read one of the images from HD and NumPy to process the data.

In [ ]:
img = cv2.imread("Exercise1/img_01_raw.png") # Read image from HD

We can then visualize images with Matplotlib.

In [ ]:
plt.imshow(img) # Display image

The result looks off, as OpenCV uses BGR color format, Matplot uses RGB. We therefore have to convert the result first:

In [ ]:
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB

plt.imshow(img_rgb) # Display result

A lot of the methods introduced in the lecture are more easily applied to grayscale images, we can also use OpenCV to convert BGR to Grayscale:

In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert BGR to grayscale

plt.imshow(img_gray) # Display result

For single channel images, a mapping to grayscale needs to be specified:

In [ ]:
plt.imshow(img_gray, cmap='gray', vmin=0, vmax=255) # Display results

As we will need this function regularly, we define a briefer alias:

In [ ]:
def show_gray(img, canvas=plt): # Later we want to draw on a different underground, so we define this as a parameter
    canvas.imshow(img, cmap='gray', vmin=0, vmax=255)

In [ ]:
show_gray(img_gray) # Use Alias

## <center><font color=navy>Histograms</font></center>
One metrics for describing images is the histogram. We can easily compute histograms using Numpy:

In [ ]:
num_values = 256 # Number of values to consider. For 8-bit 256 values are possible

histogram = np.zeros(num_values, dtype=np.int64) # Initialize histogram with zeros

In [ ]:
for i in range(num_values): # Iterate over all possible values
    histogram[i] = np.sum(img_gray == i) # Count occurences in data

In [ ]:
print(histogram)

The histogram can be more easily interpreted as a plot:

In [ ]:
plt.stairs(histogram) # Plot as stairs diagram

Generally histograms are normalized:

In [ ]:
normalized_histogram = histogram/(img_gray.shape[0]*img_gray.shape[1]) # Normalize histogram

plt.stairs(normalized_histogram)
print("Sum", np.sum(normalized_histogram))

Alternatively, OpenCV also provides a (probably faster) function for computing histograms:

In [ ]:
hist = cv2.calcHist([img_gray],[0],None,[256],[0,256]) # Calculate histogram using OpenCV
print(hist.shape) # See shape
hist = hist.reshape(-1) # Modify shape
plt.stairs(hist)

Let's combine this into one function:

In [ ]:
def calc_histogram(image):
    num_values = 256 # Number of values to consider. For 8-bit 256 values are possible

    histogram = np.zeros(num_values, dtype=np.int64) # Initialize histogram with zeros
    #TODO
    return histogram

In [ ]:
h = calc_histogram(img_gray)
plt.stairs(h)

Let's load a different image and take a look at its histogram:

In [ ]:
img_ct = cv2.imread("Exercise1/ct1.png")
img_ct = cv2.cvtColor(img_ct, cv2.COLOR_BGR2GRAY)

In [ ]:
figure, axis = plt.subplots(2, 1, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously

show_gray(img_ct, axis[0]) # the axis can be used identically to plt before

h = calc_histogram(img_ct)
axis[1].stairs(h)

We can also calculate a histogram of each channel of an RGB-image and combine them:

In [ ]:
# Calculate the histogram per channel

#TODO
figure, axis = plt.subplots(2, 1, figsize=(15, 15))

axis[0].imshow(img_rgb)

axis[1].stairs(h_r, color="r")
axis[1].stairs(h_g, color="g")
axis[1].stairs(h_b, color="b")

## <center><font color=navy>Point operations</font></center>
Point operations modify the value of a pixel, considering only its position and value, no other context, e.g.:

In [ ]:
def point_operation(image):
    img_out = np.copy(image)
    
    pos_x = np.arange(image.shape[1]).reshape(1, -1)
    pos_y = np.arange(image.shape[0]).reshape(-1, 1)
    
    pos_xy = pos_x * pos_y
    
    ind = pos_xy % 2 == 0
    img_out[ind]*= 2
    img_out[np.logical_not(ind)]*= 4
    
    return img_out

img_out = point_operation(img_gray)

figure, axis = plt.subplots(1, 2, figsize=(15, 15))

show_gray(img_gray, axis[0])
show_gray(img_out, axis[1])

Homogeneous transformations only consider the value of a pixel, e.g. affine transformations:

In [ ]:
def affine_operation(image, a, b):
    #TODO
    
    return img_out

In [ ]:
figure, axis = plt.subplots(1, 2, figsize=(15, 15))

img_res = affine_operation(img_gray, -1, 255)

show_gray(img_gray, axis[0])
show_gray(img_res, axis[1])

### <center><font color=navy>Histogram Spread</font></center>
Simple operation for increasing the contrast in a given image:

In [ ]:
def histogram_spread(image):
    #TODO
    
    return affine_operation(image, a, b)

In [ ]:
img_ct_res = histogram_spread(img_ct)

figure, axis = plt.subplots(2, 2, figsize=(15, 15))

show_gray(img_ct, axis[0, 0])
show_gray(img_ct_res, axis[0, 1])

h_ct = calc_histogram(img_ct)
h_ct_res = calc_histogram(img_ct_res)

axis[1, 0].stairs(h_ct)
axis[1, 1].stairs(h_ct_res)


### <center><font color=navy>Histogram Stretch</font></center>
Improved operation for increasing the contrast in a given image:

In [ ]:
def histogram_stretch(image, p_min=0.1, p_max=0.9):
    #TODO
    
    return affine_operation(image, a, b)

In [ ]:
img_ct_res = histogram_stretch(img_ct)

figure, axis = plt.subplots(2, 2, figsize=(15, 15))

show_gray(img_ct, axis[0, 0])
show_gray(img_ct_res, axis[0, 1])

h_ct = calc_histogram(img_ct)
h_ct_res = calc_histogram(img_ct_res)

axis[1, 0].stairs(h_ct)
axis[1, 1].stairs(h_ct_res)

### <center><font color=navy>Histogram equalization</font></center>
Further improved operation for increasing the contrast in a given image:

In [ ]:
def histogram_equalization(image):
    #TODO
    
    return img_out

In [ ]:
img_ct_res = histogram_equalization(img_ct)

figure, axis = plt.subplots(2, 2, figsize=(15, 15))

show_gray(img_ct, axis[0, 0])
show_gray(img_ct_res, axis[0, 1])

h_ct = calc_histogram(img_ct)
h_ct_res = calc_histogram(img_ct_res)

axis[1, 0].stairs(h_ct)
axis[1, 1].stairs(h_ct_res)

### <center><font color=navy>Color images</font></center>
These techniques can of course also be applied to color images:

In [ ]:
color_out = np.copy(img_rgb)

#TODO

figure, axis = plt.subplots(1, 2, figsize=(15, 15))

axis[0].imshow(img_rgb)
axis[1].imshow(color_out)

In [ ]:
color_out = np.copy(img_rgb)

#TODO

figure, axis = plt.subplots(1, 2, figsize=(15, 15))

axis[0].imshow(img_rgb)
axis[1].imshow(color_out)

In [ ]:
color_out = np.copy(img_rgb)

#TODO

figure, axis = plt.subplots(1, 2, figsize=(15, 15))

axis[0].imshow(img_rgb)
axis[1].imshow(color_out)